In [19]:
import random
import json
import pandas as pd
import os
import json

In [20]:
csv_file_path = "/Users/meetraval/Desktop/data/healthcare/Healthcare_Final_Data.csv"

model_output_path = "/Users/meetraval/Desktop/data/healthcare/processed/deepseek"


In [21]:
df = pd.read_csv(csv_file_path)
# print(len(df))

In [22]:
# Path to the folder containing the .txt files

# Create a list to store the contents in order of df index
new_column_data = []

for idx in df.index:
    file_path = os.path.join(model_output_path, f"{idx}.txt")
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            content_lst = f.read().strip().split("```json")
            if len(content_lst) > 1:
              content = json.loads(content_lst[1].split("```")[0].replace("\n", ""))
            else:
              try:
                content = json.loads(content_lst[0])
              except Exception as e:
                content_1 = """{\n  "topics":""" + content_lst[0].split("""{\n  "topics":""")[1]
                content = json.loads(content_1.replace("\n", ""))
            new_column_data.append(content)
    else:
        new_column_data.append(None)  # or '' or np.nan



In [23]:
print(len(new_column_data))
print(len(df['Conversation'].to_list()))


2288
2288


In [26]:
# Add as a new column
df['predicted_response'] = new_column_data

In [24]:
# for ii in range(len(df)):
#     pred = df['predicted_response'][ii]
#     if pred is None:
#         print(ii)
# print(df['predicted_response'][0]['topics'])

In [27]:
import ast
# Clean json responses
topics_pred, topics_true, sents_pred, sents_true = [], [], [], []
for ii in range(2082):
    json_true = ast.literal_eval(df['Filename_Json'][ii])
    json_pred = df['predicted_response'][ii]
    topic_true = list(json_true.keys())
    sent_true = list(json_true.values())
    topic_pred = [x['topic'] for x in json_pred['topics']]
    sent_pred = [x['sentiment'] for x in json_pred['topics']]
    topics_pred.append(topic_pred)
    topics_true.append(topic_true)
    sents_pred.append(sent_pred)
    sents_true.append(sent_true)

In [28]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import multilabel_confusion_matrix

# Step 1: Binarize
mlb = MultiLabelBinarizer()
y_true_bin = mlb.fit_transform(topics_true)
y_pred_bin = mlb.transform(topics_pred)

# Step 2: Compute confusion matrices
conf_matrices = multilabel_confusion_matrix(y_true_bin, y_pred_bin)

# Step 3: Print with labels
for label, matrix in zip(mlb.classes_, conf_matrices):
    print(f"Label '{label}':\n{matrix}\n")


Label 'Care Barriers':
[[1614  225]
 [ 212   31]]

Label 'Care Plan Update':
[[1606  265]
 [ 176   35]]

Label 'Diagnosis Communication':
[[1626  188]
 [ 254   14]]

Label 'Discharge Planning':
[[1368  387]
 [ 253   74]]

Label 'Documentation Review':
[[1568  242]
 [ 232   40]]

Label 'Emergency Concerns':
[[1657  243]
 [ 162   20]]

Label 'Follow Up':
[[1467  349]
 [ 177   89]]

Label 'Medical History':
[[1639  159]
 [ 168  116]]

Label 'Medication Review':
[[1502  295]
 [ 184  101]]

Label 'Patient Education':
[[1520  322]
 [ 205   35]]

Label 'Referral Coordination':
[[1605  222]
 [ 200   55]]

Label 'Risk Assessment':
[[1550  246]
 [ 241   45]]

Label 'Shared Decision':
[[1556  257]
 [ 234   35]]

Label 'Side Effects':
[[1564  274]
 [ 211   33]]

Label 'Support Services':
[[1561  363]
 [ 124   34]]

Label 'Symptom Assessment':
[[1628  198]
 [ 151  105]]

Label 'Team Communication':
[[1455  383]
 [ 178   66]]

Label 'Test Results':
[[1522  278]
 [ 154  128]]

Label 'Therapy Adherenc

/Users/meetraval/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:875: UserWarning: unknown class(es) ['Telemedicine'] will be ignored
  warnings.warn(


In [29]:
from sklearn.metrics import (
    classification_report,
    accuracy_score,
    f1_score,
    precision_score,
    recall_score
)

# --- 1. Per-Class & Aggregate Report
print(classification_report(y_true_bin, y_pred_bin, target_names=mlb.classes_))

# --- 2. Subset Accuracy (exact match)
subset_acc = accuracy_score(y_true_bin, y_pred_bin)
print(f"Subset Accuracy: {subset_acc:.4f}")

# --- 3. Micro-Averaged Scores
print("Micro Precision:", precision_score(y_true_bin, y_pred_bin, average='micro'))
print("Micro Recall:", recall_score(y_true_bin, y_pred_bin, average='micro'))
print("Micro F1 Score:", f1_score(y_true_bin, y_pred_bin, average='micro'))

# --- 4. Macro-Averaged Scores
print("Macro Precision:", precision_score(y_true_bin, y_pred_bin, average='macro'))
print("Macro Recall:", recall_score(y_true_bin, y_pred_bin, average='macro'))
print("Macro F1 Score:", f1_score(y_true_bin, y_pred_bin, average='macro'))

# --- 5. Weighted-Averaged Scores
print("Weighted Precision:", precision_score(y_true_bin, y_pred_bin, average='weighted'))
print("Weighted Recall:", recall_score(y_true_bin, y_pred_bin, average='weighted'))
print("Weighted F1 Score:", f1_score(y_true_bin, y_pred_bin, average='weighted'))

                         precision    recall  f1-score   support

          Care Barriers       0.12      0.13      0.12       243
       Care Plan Update       0.12      0.17      0.14       211
Diagnosis Communication       0.07      0.05      0.06       268
     Discharge Planning       0.16      0.23      0.19       327
   Documentation Review       0.14      0.15      0.14       272
     Emergency Concerns       0.08      0.11      0.09       182
              Follow Up       0.20      0.33      0.25       266
        Medical History       0.42      0.41      0.42       284
      Medication Review       0.26      0.35      0.30       285
      Patient Education       0.10      0.15      0.12       240
  Referral Coordination       0.20      0.22      0.21       255
        Risk Assessment       0.15      0.16      0.16       286
        Shared Decision       0.12      0.13      0.12       269
           Side Effects       0.11      0.14      0.12       244
       Support Services 

In [30]:
# Step 1: Binarize
mlb = MultiLabelBinarizer()
y_true_bin = mlb.fit_transform(sents_true)
y_pred_bin = mlb.transform(sents_pred)

# Step 2: Compute confusion matrices
conf_matrices = multilabel_confusion_matrix(y_true_bin, y_pred_bin)

# Step 3: Print with labels
for label, matrix in zip(mlb.classes_, conf_matrices):
    print(f"Label '{label}':\n{matrix}\n")

Label 'negative':
[[ 176  333]
 [ 503 1070]]

Label 'neutral':
[[297 328]
 [733 724]]

Label 'positive':
[[  38  476]
 [  98 1470]]



In [31]:
# --- 1. Per-Class & Aggregate Report
print(classification_report(y_true_bin, y_pred_bin, target_names=mlb.classes_))

# --- 2. Subset Accuracy (exact match)
subset_acc = accuracy_score(y_true_bin, y_pred_bin)
print(f"Subset Accuracy: {subset_acc:.4f}")

# --- 3. Micro-Averaged Scores
print("Micro Precision:", precision_score(y_true_bin, y_pred_bin, average='micro'))
print("Micro Recall:", recall_score(y_true_bin, y_pred_bin, average='micro'))
print("Micro F1 Score:", f1_score(y_true_bin, y_pred_bin, average='micro'))

# --- 4. Macro-Averaged Scores
print("Macro Precision:", precision_score(y_true_bin, y_pred_bin, average='macro'))
print("Macro Recall:", recall_score(y_true_bin, y_pred_bin, average='macro'))
print("Macro F1 Score:", f1_score(y_true_bin, y_pred_bin, average='macro'))

# --- 5. Weighted-Averaged Scores
print("Weighted Precision:", precision_score(y_true_bin, y_pred_bin, average='weighted'))
print("Weighted Recall:", recall_score(y_true_bin, y_pred_bin, average='weighted'))
print("Weighted F1 Score:", f1_score(y_true_bin, y_pred_bin, average='weighted'))

              precision    recall  f1-score   support

    negative       0.76      0.68      0.72      1573
     neutral       0.69      0.50      0.58      1457
    positive       0.76      0.94      0.84      1568

   micro avg       0.74      0.71      0.73      4598
   macro avg       0.74      0.70      0.71      4598
weighted avg       0.74      0.71      0.71      4598
 samples avg       0.74      0.71      0.71      4598

Subset Accuracy: 0.2397
Micro Precision: 0.7416496250852079
Micro Recall: 0.709873858199217
Micro F1 Score: 0.7254139348816536
Macro Precision: 0.73542002412152
Macro Recall: 0.7048801079850247
Macro F1 Score: 0.7109539224884464
Weighted Precision: 0.7365892596329741
Weighted Recall: 0.709873858199217
Weighted F1 Score: 0.714193584077433
